#### *ISEL - DEI - LEIM*
## Aprendizagem Automática [T52D]
### Trabalho Laboratorial 2: Classificação de Críticas de Cinema do IMDb

João Madeira ($48630$), 
Renata Góis ($51038$),
Bruno Pereira ($51811$)

**Docentes responsáveis:** 
- Prof. Gonçalo Xufre Silva

In [30]:

import numpy as np
import matplotlib.pyplot as plt
import pickle as p
import re
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score
import sklearn.preprocessing as pp
from sklearn.svm import LinearSVC

In [53]:
with open("resources/imdbFull.p", "rb") as f:
    D = p.load(f)
print("Keys:", D.keys())

reviews = D['data']
sentiments = D['target']

print(len(reviews), "reviews")

Keys: dict_keys(['data', 'target', 'DESCR'])
50000 reviews


In [25]:
def class_stats(target, predicted):
    cm = confusion_matrix(target, predicted)
    accuracy = accuracy_score(target, predicted)
    precision = precision_score(target, predicted,average="macro")
    recall = recall_score(target, predicted,average="macro")

    print("Conjunto de Teste")
    print("Accuracy:", np.round(accuracy,2))
    print("Precision:", np.round(precision,2))
    print("Recall:", np.round(recall,2))
    print(f"Matriz de confusão : \n",cm)    

This dataset contains movie reviews along with their associated binary sentiment polarity labels. It is intended to serve as a benchmark for sentiment classification. This document outlines how the dataset was gathered, and how to use the files provided.
For more details see: http://ai.stanford.edu/~amaas/data/sentiment/

### **1.** Pré-processamento e Limpeza de Texto
O objetivo desta etapa foi reduzir o ruído e a dimensionalidade do vocabulário, mantendo o conteúdo semântico relevante.

**Stemming:** Aplicou-se o algoritmo `SnowballStemmer` para reduzir as palavras à sua raiz lexical, permitindo agrupar variações da mesma palavra.

In [116]:
stemmer = SnowballStemmer("english")

def clean_review(string):
    # Remove tags HTML
    string = string.replace('<br />', ' ')  
    # Remove palavras com 20 ou mais caracteres
    string = re.sub(r'\b[a-zA-Z]{20,}\b', ' ', string)
    # Remove palavras com 3 ou mais letras repetidas consecutivamente (e.g., "yaaass", "omgggg")
    string = re.sub(r'\b\w*(.)\1{2,}\w*\b', ' ', string)
    # Filtra apenas letras
    string = re.sub(r'[^a-zA-Z]', ' ', string)
    # Remove espaços consecutivos
    string = re.sub(r'\s+', ' ', string).strip()
    # Normaliza para minúsculas
    string = string.lower()
    # Aplica Stemming
    string = " ".join(stemmer.stem(w) for w in string.split())
    return string

reviews = [clean_review(rev) for rev in reviews]

output = {"data" : reviews, "target" : sentiments}
p.dump(output,open("resources/imdbPreProcessed.p",'wb'))

### **2.** Construção do Vocabulário (TF-IDF)
Configurou-se o `TfidfVectorizer` da seguinte forma:
- **N-grams (1,2):** Incluíram-se bigramas para capturar contexto local (ex: "*not good*").

- **Stop-words:** Personalizou-se a lista de exclusão para **manter** termos de negação (como "*no*" e "*not*"), necessários para a inversão de polaridade, que seriam removidos por defeito.

- **Limites:** Definiram-se `min_df=3` e `max_features=30k` para eliminar erros ortográficos pontuais e controlar o uso de memória.

In [20]:
custom_stopwords = list(ENGLISH_STOP_WORDS - {'no', 'not', 'nor'})

tfidfVector = TfidfVectorizer(min_df=3,                    # Remove palavras que aparecem menos de 10 vezes no dataset
                        max_df=0.8,                        # Remove palavras que aparecem em 90% do dataset 
                        max_features=30000,                # Limita o maximo de features para 30.000
                        ngram_range=(1,2),                 # Utiliza unigramas e bigramas (good, very good, pretty bad)
                        token_pattern=r'\b[a-zA-Z]{2,}\b', # Ignora palavras com menos de 2 letras
                        sublinear_tf=True,                 # Term frequency passa a ter um comportamento logarítmico em vez de linear
                        stop_words=custom_stopwords        # Remove stopwords em inglês excepto "no", "not" e "nor"
                        )

In [21]:
pre_processed_data = p.load(open("resources/imdbPreProcessed.p","rb"))
reviews = pre_processed_data['data']
sentiments = pre_processed_data['target']

tfidfVector = tfidfVector.fit(reviews)

tokens = tfidfVector.get_feature_names_out()
X = tfidfVector.transform(reviews)
# X.astype(np.float32) # reduz para metade a utilização de RAM
len(tokens)

30000

### **3.** Divisão em conjuntos de treino, teste e validação
Realizou-se a partição do *dataset* em três subconjuntos: **treino**, **teste** e **validação**. Utilizou-se amostragem estratificada (`stratify`) para garantir que a distribuição original das classes de *rating* ($1$ a $10$) fosse preservada proporcionalmente em todos os subconjuntos.

| Treino | Teste | Validação |
|--|--|--|
|40k (80%)|5k (10%)|5k (10%)|

In [22]:
# Split 80% treino / 20% temporário (validação + teste)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, sentiments,
    test_size=0.2,
    random_state=42,
    stratify=sentiments
)

# Split 50% validação, 50% teste (10% / 10%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

### **4.** Seleção de Modelo e Hiperparâmetros
Optou-se por testar os modelos **LogisticRegression**, **LinearSVC** e **Random Forest**

### LogisticRegression

* **Otimização:** Efetuou-se uma pesquisa em grelha (`GridSearchCV`) com validação cruzada (`cv=3`) para determinar o valor ideal do parâmetro de regularização `C`.

* **Regularização:** Manteve-se a penalização L2 para mitigar o risco de *overfitting*.

In [115]:
param_grid = {
    'penalty': ['l2'],
    'solver': ['saga'],
    'C': [1, 2, 5],
    'max_iter': [50,100]
}

grid_search = GridSearchCV(LogisticRegression(random_state=42, n_jobs=1), param_grid,cv=3)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'C': 1, 'max_iter': 50, 'penalty': 'l2', 'solver': 'saga'}


In [26]:
lr = LogisticRegression(penalty='l2', solver='saga', C=1, max_iter=50, random_state=42)
lr = lr.fit(X_train, y_train)

### Análise de Desempenho
Avaliou-se o classificador com os conjuntos de Teste e Validação.

In [ ]:
predicted_lrtest = lr.predict(X_test)

class_stats(y_test, predicted_lrtest, lr.predict_proba(X_test))

Conjunto de Teste
número de erros : 2836
percentagem de acertos : 43.28%
Matriz de confusão : 
[[828  47  47  40   7  10   3  30]
 [278  28  50  63   8  11   3  18]
 [200  23  81 102  17  27   7  39]
 [140  24  65 163  47  22   7  65]
 [ 28   6  11  41 121 124  19 130]
 [ 19   4  10  31  98 159  21 244]
 [ 10   1   6   8  38  91  31 276]
 [ 46   1   8   5  36  95  29 753]]


In [ ]:
predicted_lrval = lr.predict(X_val)

class_stats(y_val, predicted_lrval, lr.predict_proba(X_val))

Conjunto de Validação
número de erros : 2832
percentagem de acertos : 43.36%
Matriz de confusão : 
[[817  34  50  56   6  10   5  34]
 [266  35  42  72  10   9   2  22]
 [189  39  83 114  23  16   5  27]
 [129  19  71 188  51  30   7  38]
 [ 29   8  21  41 129 111  15 127]
 [ 26   5   6  26 106 126  39 252]
 [ 13   2   6   8  33  87  35 277]
 [ 40   2   5  16  32  93  30 755]]


### LinearSVC

In [ ]:
param_grid = {
    'C': [0.1, 1, 2, 5],
    'max_iter': [2000, 5000, 10000]
}

grid_search = GridSearchCV(LinearSVC(random_state=42), param_grid,cv=3)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)


In [23]:
lsvc = LinearSVC(random_state=42, C=0.1, max_iter=2000)
lsvc = lsvc.fit(X_train, y_train)

### Análise de Desempenho

In [28]:
predicted_svctest = lsvc.predict(X_test)

class_stats(y_test, predicted_svctest)

Conjunto de Teste
Accuracy: 0.43
Precision: 0.34
Recall: 0.33
Matriz de confusão : 
 [[869  25  32  32   5   9   3  37]
 [312  22  31  55   8  10   2  19]
 [228  21  61  95  21  18   5  47]
 [172  18  55 151  38  22   8  69]
 [ 30   6  12  40 108 108  15 161]
 [ 24   3   7  28  90 122  14 298]
 [ 17   1   1   7  34  63  24 314]
 [ 44   1   5   7  29  70  18 799]]


In [29]:
predicted_svcval = lsvc.predict(X_val)

class_stats(y_val, predicted_svcval)

Conjunto de Teste
Accuracy: 0.44
Precision: 0.34
Recall: 0.33
Matriz de confusão : 
 [[864  24  27  39   6   9   3  40]
 [307  20  25  62   9   6   1  28]
 [231  22  71 108  17  13   4  30]
 [150  17  67 172  44  23   8  52]
 [ 32   4  14  40 119 107   7 158]
 [ 30   3   9  22  96 115  23 288]
 [ 15   2   6  10  28  67  21 312]
 [ 36   1   3  12  27  67  22 805]]


In [ ]:
param_grid = {
    "n_estimators": [100, 200],
    "max_depth": [None],
    "min_samples_split": [2]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid,cv=3)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
# não foi possivel obter um resultado com o grid search

In [38]:
rfc = RandomForestClassifier(random_state=42, n_estimators= 200, max_depth= None, min_samples_split= 2)
rfc = rfc.fit(X_train, y_train)

### Análise de Desempenho

In [39]:
predicted_rfctest = rfc.predict(X_test)

class_stats(y_test, predicted_rfctest)

Conjunto de Teste
Accuracy: 0.38
Precision: 0.46
Recall: 0.25
Matriz de confusão : 
 [[916   0   2   6   2   5   0  81]
 [382   5   2   5   2   3   0  60]
 [346   0  12  18   2   5   0 113]
 [302   0   9  29   5  14   0 174]
 [101   0   1  14  28  37   0 299]
 [ 81   0   4   7  15  42   0 437]
 [ 38   0   1   6   5  23   2 386]
 [ 83   0   0   2   7  19   2 860]]


In [40]:
predicted_rfctest = rfc.predict(X_val)

class_stats(y_val, predicted_rfctest)

Conjunto de Teste
Accuracy: 0.39
Precision: 0.53
Recall: 0.26
Matriz de confusão : 
 [[917   0   1   5   0   3   0  86]
 [363   6   4  10   0   1   0  74]
 [365   0  17  17   3   6   0  88]
 [323   0   6  41   8  18   0 137]
 [ 94   0   4  18  30  49   1 285]
 [ 95   0   2   7  16  49   0 417]
 [ 38   0   0   2   7  24   8 382]
 [ 84   0   0   3   6  22   0 858]]
